In [2]:
import importlib
import inv
import labs
import preds

importlib.reload(labs)
importlib.reload(preds)
importlib.reload(inv)

from labs import *
from inv import *
from preds import *

label2id, id2label = make_doc_class_mapper('/data/ephemeral/home/dataset/dtc/doc_classes.json')
id2kor = make_id2kor(id2label)

# 클래스 정리 (훈련 기준)

#### 아주 잘 분류됨 (7종)
- 계좌번호 (0) , 차량 계기판 (2) ,운전면허증 (5) , 민증 (8) , 여권 (9) , 차량등록증 (15) , 자동차 (16)

#### 꽤 잘됨 (4종)
- 의료비 영수증 (6)  , 납입 확인서 (1) , 약국 영수증 (11) , 처방전 (12)

#### 잘 안됨 (6종)
- 임신/출산 (1) , 입퇴원 확인서 (3) , 진단서 (4) , 외래 진료/통원 확인서 (7) , 이력서 (13) , 소견서 (14)

- 최악: 입퇴원 / 외래 / 진단서 
- 보통: 이력서 / 임신 / 소견서

### NOTE
- 입퇴원 확인서와 외래/진료/통원 확인서는 확인서라는 단어에 의해 서로 혼동이 심한편
- 진단서와 진료확인서는 '진' 뒷글자가 없는 경우 혼동

# Analysis

# load history

In [3]:
path_csvs = grep_files('./history/', exts=['csv'])

hists = []
for p in path_csvs: 
    df = pd.read_csv(p)
    hists.append(df)

0it [00:00, ?it/s]

In [4]:
def calculate_match_rate(dfs, target_col):
    targets = pd.concat([df[target_col].reset_index(drop=True) for df in dfs], axis=1)
    match = targets.nunique(axis=1) == 1
    match_rate = match.mean()
    return match_rate

In [5]:
calculate_match_rate(hists, 'target')

0.8697452229299363

In [6]:
import itertools
def partial_match_rate(dfs, target_col):
    targets = pd.concat([df[target_col].reset_index(drop=True) for df in dfs], axis=1)
    n = len(dfs)
    total_pairs = n * (n - 1) // 2

    def match_rate(row):
        matches = sum(1 for a, b in itertools.combinations(row, 2) if a == b)
        return matches / total_pairs if total_pairs > 0 else 1.0

    rates = targets.apply(match_rate, axis=1)
    return rates, rates.mean()

In [8]:
rates, means = partial_match_rate(hists, 'target')
print(means)

0.9350106157112527


In [9]:
wrongs = hists[0][rates < 1.0]['ID'].tolist()
write_list('inconsist_list.txt', wrongs)

In [10]:
base_path = '/data/ephemeral/home/dataset/dtc/test/'
wrong_paths = [os.path.join(base_path, n) for n in wrongs]

# 살펴보기

In [84]:
hists[0].iloc[1]['target']

12

In [105]:
hists[0].index[hists[0]['ID'] == '0008fdb22ddce0ce.jpg']

Index([0], dtype='int64')

In [88]:
inv = ImageNavigator(image_paths=wrong_paths, hists=hists, id2kor=id2kor)

# Voting

In [11]:
def vote_df(dfs, target_col):
    df_ans = new_df = dfs[0].copy()
    targets = pd.concat([df[target_col].reset_index(drop=True) for df in dfs], axis=1)
    voted = targets.mode(axis=1)[0]
    df_ans[target_col] = voted.astype(int)
    return df_ans

In [12]:
df_voted = vote_df(hists, 'target')

In [99]:
df_voted.to_csv('voted.csv', index=False)

# Fixed List 만들기

In [47]:
# df = df_voted
# df.loc[df['ID'].isin(wrongs), 'target'] = -1
# df.to_csv('fixed.csv', index=False)

# 테스트셋 fixed로 분포보기

In [51]:
df['class'] = df['target'].map(id2kor)

In [58]:
df['class'].value_counts()

class
계기판          200
차량등록증        200
자동차          200
여권           200
주민등록증        199
계좌번호         198
면허증          197
처방전          196
의료비 영수증      196
납입 확인서       191
약국 영수증       185
이력서          139
진단서          134
입퇴원 확인서       97
임신/출산         88
진료/통원 확인서     69
소견서           42
Name: count, dtype: int64

In [66]:
ps = grep_files('/data/ephemeral/home/dataset/docsy-deskew/', exts=['jpg'])
print(len(ps))

0it [00:00, ?it/s]

3140


# 훈련 데이터 보기

In [68]:
ps = grep_files('/data/ephemeral/home/dataset/dtc/train/', exts=['jpg'])
len(ps)

0it [00:00, ?it/s]

1570

In [70]:
inv = ImageNavigator(ps, [df], id2kor)